In [2]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup as bs

In [93]:
def scrape_zongheng(url):
    response = requests.get(url)
    soup = bs(response.content, "html.parser")
    title = soup.find("div", attrs={"class": "book-info--title"}).get_text(strip=True)
    div_tags = soup.find("div", attrs={"class": "book-info--tags"})
    tags = []
    serial_status = None
    category = None
    for span in div_tags.find_all("span"):
        try:
            if span["class"][0] == "serialStatus":
                # serial_status = "ongoing" if span.get_text(strip=True) == "连载中" else "completed"
                serial_status = span.get_text(strip=True)
                tags.append(serial_status)
            elif span["class"][0] == "cateFineId":
                category = span.get_text(strip=True)
                tags.append(category)
        except KeyError:
            tags.append(span.get_text(strip=True))
    recent_chapter_info = soup.find(
        "div", attrs={"class": "book-info--chapter-name"}
    ).find(
        "a", attrs={"class": "global-hover"}
    )
    word_count = recent_chapter_info["title"].split("字数：")[1].split()[0]
    author = soup.find("a", attrs={"class": "author-info--name"}).get_text(strip=True)
    return {
        "title": title,
        "tags": tags,
        "serial_status": serial_status,
        "category": category,
        "word_count": word_count,
        "author": author,
    }

scrape_zongheng("https://www.zongheng.com/detail/325639")

{'title': '星辰之主',
 'tags': ['连载中', '科幻', '星空', '未来高武', '伪科幻'],
 'serial_status': '连载中',
 'category': '科幻',
 'word_count': '7498927',
 'author': '减肥专家'}

In [111]:
def scrape_jjwxc(url):
    response = requests.get(url)
    soup = bs(response.content, "html.parser")
    title = soup.find("span", attrs={"itemprop": "articleSection"}).get_text(strip=True)
    tags = soup.find("span", attrs={"itemprop": "genre"}).get_text(strip=True).split("-")
    serial_status = soup.find("span", attrs={"itemprop": "updataStatus"}).get_text(strip=True)
    word_count = int(soup.find("span", attrs={"itemprop": "wordCount"}).get_text(strip=True)[:-1])
    tags.append(serial_status)
    extra_tags = soup.find_all("div", attrs={"class": "smallreadbody"})
    for div in extra_tags:
        if div.find("span") is not None:
            extra = div.find_all("a", attrs={"style": "text-decoration:none;color: red;"})
            for a in extra:
                tags.append(a.get_text(strip=True))
    author = soup.find("span", attrs={"itemprop": "author"}).get_text(strip=True)
    return {
        "title": title,
        "tags": tags,
        "serial_status": serial_status,
        "category": None,
        "word_count": word_count,
        "author": author,
    }

scrape_jjwxc("https://www.jjwxc.net/onebook.php?novelid=6079968")

{'title': '见春天',
 'tags': ['原创', '言情', '近代现代', '爱情', '完结', '花季雨季', '情有独钟', '阴差阳错', '校园', '正剧'],
 'serial_status': '完结',
 'category': None,
 'word_count': 191449,
 'author': '纵虎嗅花'}

In [ ]:
def scrape_17k(url):
    response = requests.get(url)
    soup = bs(response.content, "html.parser")

In [ ]:
def scrape_qidian(url):
    response = requests.get(url)
    soup = bs(response.content, "html.parser")